# Defense of the Ancients 2 Outcome Prediction Modelling

In [1]:
import pandas as pd
import pickle
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import make_scorer
from sklearn import metrics
from copy import deepcopy
from sqlalchemy import create_engine
import os

/home/paul/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


**INFORMATION ON DATASETS PROVIDED BY KAGGLE:**

**matches: **contains top level information about each match. see https://wiki.teamfortress.com/wiki/WebAPI/GetMatchDetails#Tower_Status%22tower_status_dire%22:%202047) for interpreting tower and barracks status. Cluster can link matches to geographic region.  
**players:** Individual players are identified by account_id but there is an option to play anonymously and roughly one third of the account_id are not available. Anonymous users have the value of 0 for account_id. Contains totals for kills, deaths, denies, etc. Player action counts are available, and are indicated by variable names beginning with unit_order_. Counts for reasons for acquiring or losing gold, and gaining experience, have prefixes gold_, and xp_.  
**player_time:** Contains last hits, experience, and gold sampled at one minute interval for all players in all matches. The column names indicate the player_slot. For instance xp_t_1 indicates that this column has experience sums for the player in slot one.  
**teamfights:** Start and stop time of teamfights, as well as last death time. Teamfights appear to be all battles with three or more deaths. As such this does not include all battles for the entire match.  
**teamfights_players :** Additional information provided for each player in each teamfight. player_slot can be used to link this back to players.csv
objectives: Gives information on all the objectives completed, by which player and at what time.  
**chat:** All chat for the 50k matches. There is plenty of profanity, and good natured trolling.   


**purchase_log** item purchase times  
**ability_upgrade** ability upgrade times and levels 
  
  
patch_dates release dates for various patches, use start_time from match.csv to determine which patch a match was played in.  
  
  
ability_ids use with ability_upgrades.csv to get the names of upgraded abilities  
item_ids use with purchase_log.csv to get the names of purchased items  

## Data Cleaning

### Functions

In [3]:
def team_assign(x):
    """Assigns a team value to each player of either team 0 or team 1
    the dataframe identifies each player individually.
    team 0 player_ids: 1-5 | team 1 player_ids: 111-115
    For data flexibility purposes the teams are internally 0 and 1, but
    future column naming conventions with refer to them as team 1 and 
    team 2 respectively"""
    if x < 100:
        y = 0
    else:
        y = 1
    return y

In [4]:
def get_sum_players(df, column):
    """Returns the sum of column 'column' for each team in each match as a new column for Dataframe players"""
    temp_df = pd.DataFrame()
    # group by match_id and team
    temp_df[column] = players.groupby(['match_id','team'])[column].sum()
    temp_df = temp_df.reset_index()
    # split into two dataframes
    temp_df1 = temp_df.query('team == 0')
    temp_df2 = temp_df.query('team == 1')
    temp_df1.drop('team',axis=1,inplace=True)
    temp_df2.drop('team',axis=1,inplace=True)
    # merge back to original dataframe with suffixes based on team id.
    temp_df = temp_df1.merge(temp_df2, how='left', on='match_id', suffixes = ['_1','_2'])
    return df.merge(temp_df, on='match_id', how='left')

In [5]:
def get_mean_players(df, column):
    """Returns the mean of column 'column' for each team in each match as a new column for Dataframe players"""
    temp_df = pd.DataFrame()
    temp_df[column] = players.groupby(['match_id','team'])[column].mean()
    temp_df = temp_df.reset_index()
    temp_df1 = temp_df.query('team == 0')
    temp_df2 = temp_df.query('team == 1')
    temp_df1.drop('team',axis=1,inplace=True)
    temp_df2.drop('team',axis=1,inplace=True)
    temp_df = temp_df1.merge(temp_df2, how='left', on='match_id', suffixes = ['_1','_2'])
    return df.merge(temp_df, on='match_id', how='left')

In [6]:
def get_std_players(df, column):
    """Returns the standard deviation of column 'column' for each team in each match as a new column for Dataframe players.
    This was not used in the final procduct, but may be picked up later"""
    temp_df = pd.DataFrame()
    temp_df[column] = players.groupby(['match_id','team'])[column].std()
    temp_df = temp_df.reset_index()
    temp_df1 = temp_df.query('team == 0')
    temp_df2 = temp_df.query('team == 1')
    temp_df1.drop('team',axis=1,inplace=True)
    temp_df2.drop('team',axis=1,inplace=True)
    temp_df = temp_df1.merge(temp_df2, how='left', on='match_id', suffixes = ['_1','_2'])
    return df.merge(temp_df, on='match_id', how='left')

In [7]:
def get_mean_teamfights(df, column):
    """Returns the mean of column 'column' for each team for each teamfight in each match as a new column for Dataframe teamfights."""
    temp_df = pd.DataFrame()
    temp_df2 = pd.DataFrame()
    temp_df[column] = teamfights_players.groupby(['match_id','team','team_fight'])[column].mean()
    temp_df = temp_df.reset_index()
    temp_df2[column] = temp_df.groupby(['match_id','team'])[column].mean()
    temp_df2 = temp_df2.reset_index()
    temp_df3 = temp_df2.query('team == 0')
    temp_df4 = temp_df2.query('team == 1')
    temp_df3.drop('team',axis=1,inplace=True)
    temp_df4.drop('team',axis=1,inplace=True)
    temp_df = temp_df3.merge(temp_df4, how='left', on='match_id', suffixes = ['_1','_2'])
    return df.merge(temp_df, on='match_id', how='left')

In [8]:
def get_std_teamfights(df, column):
    """Returns the standard deviation of column 'column' for each team for each teamfight in each match as a new column for Dataframe teamfights.
    This was not used in the final procduct, but may be picked up later"""
    temp_df = pd.DataFrame()
    temp_df2 = pd.DataFrame()
    temp_df[column] = teamfights_players.groupby(['match_id','team','team_fight'])[column].std()
    temp_df = temp_df.reset_index()
    temp_df2[column] = temp_df.groupby(['match_id','team'])[column].mean()
    temp_df2 = temp_df2.reset_index()
    temp_df3 = temp_df2.query('team == 0')
    temp_df4 = temp_df2.query('team == 1')
    temp_df3.drop('team',axis=1,inplace=True)
    temp_df4.drop('team',axis=1,inplace=True)
    temp_df = temp_df3.merge(temp_df4, how='left', on='match_id', suffixes = ['_1','_2'])
    return df.merge(temp_df, on='match_id', how='left')

### Match Outcome Data

In [9]:
password = os.environ['POSTGRESQL_PASS']
ip_address = os.environ['AWS_IP']

In [10]:
engine = create_engine('postgresql://ubuntu:%s@%s:5432/' % (password, ip_address))

In [11]:
conn = engine.connect()

In [12]:
match = pd.read_sql_query('''SELECT match_id, game_mode, radiant_win FROM match''', engine)

In [13]:
teamfights_players = pd.read_sql_query('''SELECT match_id, player_slot, damage FROM teamfights_players''', engine)

In [14]:
players = pd.read_sql_query('''SELECT match_id, player_slot, leaver_status, gold_spent, gold, level, xp_per_min, hero_healing, kills, deaths, \
assists FROM players''', engine)

In [15]:
conn.close()

In [16]:
players;

In [17]:
# Generate team column for grouping by player team
players['team'] = players['player_slot'].apply(team_assign)

In [18]:
# Switch win status from True False to values 0 and 1 and change name to winner
match['winner'] = match['radiant_win'].apply(lambda x: 1 if x == True else 0)
match.drop('radiant_win',axis=1,inplace=True)

In [19]:
# Check to see if any players left early or were disconnected from the match
match['leaver_status'] = players.groupby(['match_id'])['leaver_status'].sum()

In [20]:
# Calculate individual percentage gold spent by player_id
players['gold_spent_percentage'] = players['gold_spent'] / (players['gold'] + players['gold_spent'])

In [21]:
# Generate a list of columns for iterating through and gathering team mean or team sum
players_mean_list = ['gold_spent','gold_spent_percentage','level','xp_per_min','hero_healing']
players_sum_list = ['kills','deaths','assists']
master_col_list = players_mean_list + players_sum_list

In [22]:
for col_name in players_mean_list:
    match = get_mean_players(match, col_name)
for col_name in players_sum_list:
    match = get_sum_players(match, col_name)

/home/paul/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/paul/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/paul/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/paul/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a co

In [23]:
# Convert total deaths per team to just environmentally caused deaths
match['env_deaths_1'] = match['deaths_1'] - match['kills_2']
match['env_deaths_2'] = match['deaths_2'] - match['kills_1']
master_col_list.append('env_deaths')
master_col_list.remove('deaths')

In [24]:
match['asst_rate_1'] =  match['assists_1'] / match['kills_1']
match['asst_rate_2'] =  match['assists_2'] / match['kills_2']

match['asst_rate_2'] = match['asst_rate_2'].apply(lambda x: 0.0 if x == np.inf else x)
match['asst_rate_1'] = match['asst_rate_1'].apply(lambda x: 0.0 if x == np.inf else x)

master_col_list.append('asst_rate')
master_col_list.remove('assists')

### Team Fights Data

In [25]:
teamfights_players['team'] = teamfights_players['player_slot'].apply(team_assign)

In [26]:
teamfights_players['team_fight'] = [x//10 for x in range(len(teamfights_players))]

In [27]:
def binarize(x):
    """Used for determining if a player did or did not cause damage in a team fight as a proxy for participation"""
    if x > 0:
        x = 1
    else:
        x = 0
    return x

In [28]:
teamfights_players['damage'] = teamfights_players['damage'].apply(binarize)

In [29]:
match = get_mean_teamfights(match, 'damage')

/home/paul/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/paul/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [30]:
master_col_list.append('damage')

In [31]:
# Only look at games in which no players left and the games were ranked
match = match.query('leaver_status == 0').query('game_mode == "22"')

In [32]:
match = match.fillna(0)

In [33]:
X = match.iloc[:,4:]
y = match.iloc[:,2]

In [34]:
X.shape

(41047, 22)

In [35]:
xscaled = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(xscaled,y,test_size=0.3, random_state=42)
rfc = RandomForestClassifier(max_depth=9)
rfc.fit(X_train,y_train)
rfc_acc = rfc.score(X_test,y_test)

print(rfc_acc)

0.976938692651


## Advantage DataFrame and RFC Modeling

In [36]:
# For computational speed look at team 1's advantage/disadvantage when compared to team two and reduce 18 features down to 9
advantage_df = pd.DataFrame()

for col_name in master_col_list:
    advantage_df[col_name] = match['%s_%s' % (col_name, '1')] - match['%s_%s' % (col_name, '2')]

In [37]:
advantage_df['winner'] = match['winner']

In [38]:
advantage_df.rename(columns={'damage':'teamfight_participation'}, inplace = True)

In [39]:
y = advantage_df.iloc[:,-1]

X = advantage_df.iloc[:,:-1]

In [40]:
xscaled = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(xscaled,y,test_size=0.3, random_state=42)
rfc = RandomForestClassifier(max_depth=9)
rfc.fit(X_train,y_train)
rfc_acc = rfc.score(X_test,y_test)

print(rfc_acc)

0.985708485587


### Make all combinations of features

In [41]:
import itertools
# all unique combinations of on/off for features (columns) in x
comb_list = list(itertools.product([0, 1], repeat=len(X.iloc[0])))
# 
comb_ind = []

for x in comb_list:
    indices = [i for i, v in enumerate(x) if v == 1]
    comb_ind.append(indices)

In [43]:
comb_list[0]

(0, 0, 0, 0, 0, 0, 0, 0, 0)

In [44]:
# translator for converting binary combiation lists into a unique value 
base_10_translator = [2**x for x in range(len(X.iloc[0]))]
base_10_translator = base_10_translator[::-1]

In [46]:
# remove combination in which no features are used
comb_list.remove((0, 0, 0, 0, 0, 0, 0, 0, 0))

comb_ind.remove([])

In [ ]:
# Generate dictionary for holding all of the 

model_dict = {}

In [ ]:
# Grid Search CV Params 
max_depth_range = ([5,10,15,20])
n_estimators =[50,100,500,1000]
param_grid = dict(max_depth=max_depth_range, n_estimators=n_estimators) 
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

In [47]:
# first pass checking for best params
for k, v in enumerate(comb_list):
    xscaled = StandardScaler().fit_transform(X.iloc[:,comb_ind[k]])
    X_train, X_test, y_train, y_test = train_test_split(xscaled,y,test_size=0.3, random_state=42)
    grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid=param_grid, cv=cv)
    grid.fit(X_train, y_train)
    y_pred=grid.predict(X_test)
    params = grid.best_params_
    scores = {}
    scores['code'] = np.dot(v, base_10_translator)
    scores['accuracy'] = metrics.accuracy_score(y_test,y_pred)
    scores['precision'] = metrics.precision_score(y_test,y_pred)
    scores['recall'] = metrics.recall_score(y_test,y_pred)
    scores['f1_score'] = metrics.f1_score(y_test,y_pred)

    model_dict[np.dot(v, base_10_translator)] = dict(scores = scores, params = params);

In [48]:
# didn't end up saving best params, but this can be done at a later date with this template
df = pd.DataFrame([], columns = ['Group','Scores'])
for k,v in enumerate(comb_list):
    temp = X.iloc[:,comb_ind[k]]
    xscaled = StandardScaler().fit_transform(temp)
    X_train, X_test, y_train, y_test = train_test_split(xscaled,y,test_size=0.3, random_state=42)
    grid = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv=cv)
    grid.fit(X_train, y_train)
    y_pred=grid.predict(X_test)
    df2 = pd.DataFrame([[np.dot(v, base_10_translator), metrics.accuracy_score(y_test,y_pred)],
                        [np.dot(v, base_10_translator),metrics.precision_score(y_test,y_pred)],
                         [np.dot(v, base_10_translator),metrics.recall_score(y_test,y_pred)],
                         [np.dot(v, base_10_translator),metrics.f1_score(y_test,y_pred)]], columns=['Group','Scores'])
    df = df.append(df2, ignore_index=True);

In [8]:
# Reposition data for vertical bar chart in visualization
vbar_df = deepcopy(df)
vbar_df['Accuracy'] = vbar_df['Scores']

vbar_df['Precision'] = vbar_df['Scores'].shift(-1)

vbar_df['Recall'] = vbar_df['Scores'].shift(-2)

vbar_df['F1 score'] = vbar_df['Scores'].shift(-3)

vbar_df.drop('Scores', axis=1, inplace=True)

vbar_df.drop_duplicates('Group', inplace = True)

nums = [1,2,4,8,16,32,64,128, 256]

vbar_df = vbar_df.query('Group in @nums')

vbar_df = vbar_df.reset_index()

vbar_df.drop("index", axis=1, inplace = True)

vbar_df.to_csv('./viz/vbardata.csv')

In [216]:
# Prepare data for horizontal bar chart 

hbar_df = deepcopy(df)

hbar_df = hbar_df.groupby(['Group'])['Scores']

hbar_df = hbar_df.reset_index()

hbar_df.drop("index", axis=1, inplace = True)

hbar_df.to_csv('./viz/vbardata.csv')

In [79]:
pickle.dump( model_dict, open( "model_dict.p", "wb" ) )

In [ ]:
# Search for best overall combination

df['mean'] = (df.Accuracy + df.Precision + df.Recall + df.f1score)/4

df['mean'].max()

In [19]:
df.query('mean == 0.98612175384788592')

,Group,Accuracy,Precision,Recall,f1score,mean
1532,384.0,0.985627,0.985564,0.986648,0.986648,0.986122
